In [3]:
! pip install tensorflow==2.9.2 tensorflow-gpu==2.9.2 opencv-python mediapipe sklearn matplotlib

  Using cached tensorflow-2.9.2-cp39-cp39-win_amd64.whl (444.1 MB)
     ------------------------------------ 444.1/444.1 MB 664.0 kB/s eta 0:00:00
  Using cached opencv_python-4.6.0.66-cp36-abi3-win_amd64.whl (35.6 MB)
  Using cached mediapipe-0.8.11-cp39-cp39-win_amd64.whl (49.0 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
     ---------------------------------------- 1.5/1.5 MB 1.4 MB/s eta 0:00:00
  Using cached absl_py-1.3.0-py3-none-any.whl (124 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
     ---------------------------------------- 2.6/2.6 MB 2.3 MB/s eta 0:00:00
  Using cached tensorboard-2.9.1-py3-none-any.whl (5.8 MB)
     -------------------------------------- 895.9/895.9 kB 2.8 MB/s eta 0:00:00
  Using cached tensorflow_estimator-2.9.0-py2.py3-none-any.whl (438 kB)
  Using cached Keras_

In [5]:
import cv2 as cv
import numpy as np
import os
import time
from matplotlib import pyplot as plt
import mediapipe as mp

%matplotlib inline

In [6]:
cap = cv.VideoCapture(0)
while cap.isOpened():
    ret,frame = cap.read()
    cv.imshow('OpenCV Feed', frame)
    if cv.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv.destroyAllWindows()

### Keypoints using Mediapipe Holistic

In [ ]:
mp_holistic = mp.